In [ ]:
import torch
import os

# https://www.youtube.com/watch?v=E5kzAbD8D0w, DPO
# https://www.youtube.com/watch?v=wybjsKtA9hI, SQL Backend
# https://www.youtube.com/watch?v=CZw57SIwgiE, Threads and Connections
# https://www.youtube.com/watch?v=p6wwj0ozifw, Hashing
# https://www.youtube.com/watch?v=4NB0NDtOwIQ, sys_design
# https://www.youtube.com/watch?v=kxT8-C1vmd4, zig
# https://www.youtube.com/watch?v=5C_HPTJg5ek, rust
# https://www.youtube.com/watch?v=HyWYpM_S-2c, React
# https://www.youtube.com/watch?v=MnpuK0MK4yo, react_svelte
# https://www.youtube.com/watch?v=mE7IDf2SmJg, stanford_lecture_rag
# https://www.youtube.com/watch?v=utYC2tKDdoQ, umich_os_locks
# https://www.youtube.com/watch?v=YAgjfMR9R_M, umich_dl_attention
# https://www.youtube.com/watch?v=dsHyUgGMht0, k_fang_github_break 


videos = [
    ["https://www.youtube.com/watch?v=UiX8K-xBUpE", "UiX8K-xBUpE", "mixtral8x7b"],
    ["https://www.youtube.com/watch?v=Xdhe-meS30A", "Xdhe-meS30A", "catan"],
    ["https://www.youtube.com/watch?v=00AQjBDM46Q", "00AQjBDM46Q", "mlops_llm_eval"],
    ["https://www.youtube.com/watch?v=5ChkQKUzDCs", "5ChkQKUzDCs", "typescript_fireship"],
    ["https://www.youtube.com/watch?v=J0NuOlA2xDc", "J0NuOlA2xDc", "localized_deployment"],
    ["https://www.youtube.com/watch?v=E5kzAbD8D0w", "E5kzAbD8D0w", "dpo"],
    ["https://www.youtube.com/watch?v=wybjsKtA9hI", "wybjsKtA9hI", "sql_backend"],
    ["https://www.youtube.com/watch?v=CZw57SIwgiE", "CZw57SIwgiE", "threads_connections"],
    ["https://www.youtube.com/watch?v=p6wwj0ozifw", "p6wwj0ozifw", "hashing"],
    # ["https://www.youtube.com/watch?v=4NB0NDtOwIQ", "4NB0NDtOwIQ", "sys_design"], // Note this failed due to not having a timestamp every line
    ["https://www.youtube.com/watch?v=kxT8-C1 vmd4", "kxT8-C1vmd4", "zig"],
    ["https://www.youtube.com/watch?v=5C_HPTJg5ek", "5C_HPTJg5ek", "rust"],
    ["https://www.youtube.com/watch?v=HyWYpM_S-2c", "HyWYpM_S-2c", "react"],
    ["https://www.youtube.com/watch?v=MnpuK0MK4yo", "MnpuK0MK4yo", "react_svelte"],
]

name_to_url = {
    "mixtral8x7b": "https://www.youtube.com/watch?v=UiX8K-xBUpE",
    "catan": "https://www.youtube.com/watch?v=Xdhe-meS30A",
    "mlops_llm_eval": "https://www.youtube.com/watch?v=00AQjBDM46Q",
    "typescript_fireship": "https://www.youtube.com/watch?v=5ChkQKUzDCs",
    "localized_deployment": "https://www.youtube.com/watch?v=J0NuOlA2xDc",
    "dpo": "https://www.youtube.com/watch?v=E5kzAbD8D0w",
    "sql_backend": "https://www.youtube.com/watch?v=wybjsKtA9hI",
    "threads_connections": "https://www.youtube.com/watch?v=CZw57SIwgiE",
    "hashing": "https://www.youtube.com/watch?v=p6wwj0ozifw",
    "zig": "https://www.youtube.com/watch?v=kxT8-C1vmd4",
    "rust": "https://www.youtube.com/watch?v=5C_HPTJg5ek",
    "react": "https://www.youtube.com/watch?v=HyWYpM_S-2c",
    "react_svelte": "https://www.youtube.com/watch?v=MnpuK0MK4yo",
}

In [ ]:
from video import download_video
from transcript import download_transcript

# download_video('https://www.youtube.com/watch?v=Xdhe-meS30A', 'data/videos/video.mp4')
for link, video_id, name in videos:
    download_transcript(video_id, f'data/transcripts/raw/{name}.txt')
    ## Clean
    print(name)

    with open(f'data/transcripts/raw/{name}.txt', 'r') as file:
        with open(f'data/transcripts/clean/{name}.txt', 'w') as file_clean:
            for i,line in enumerate(file):

                line = line.split(',')[2]
                ## Remove newline
                line = line.replace('\n', ' ')
                if i != 0:
                    file_clean.write(line)
            

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:

# Prepare the input text and tokenize
## read from transcript.txt
# name, input
inputs_dict = {}

# open all files in data/transcripts/clean
for file in os.listdir('data/transcripts/clean'):
    with open(f'data/transcripts/clean/{file}') as f:
        input_text = f.read()[0:4000]
        inputs_dict[file[:-4]] = tokenizer(input_text, return_tensors="pt")

for name, inputs in inputs_dict.items():
    print(f'Filename: {name}, Tokens: {len(inputs[0].tokens)}')

In [ ]:
### catan.txt
## Clean transcript, with new lines
## Char: 9388, Word: 1840, Tokens: 2483, Runtime: 3m 4.2s on Pytorch MPS, T/S = 13.5

## Clean transcript, no new lines
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 43.9s on Pytorch MPS, T/S = 13.25

## Clean transcript, charging
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 32.8s on Pytorch MPS

#### On A10s
# 8,000-token limit for Mistral-7B

### catan.txt
## Tokens: 2174, Runtime: 48.8s, T/S = 44.55

### mixtral8x7b.txt
## Tokens: 16981, Runtime: 13m 8.4s, T/S = 21.59

### mlops_llm_eval.txt
## Tokens: 10993, Runtime: 7m 5.1s, T/S = 25.87

### typescript_fireship.txt
## Tokens: 1042, Runtime: 27.6s, T/S = 37.75

### localized_deployment.txt | Tokens: 892,
## A10 // Runtime: 21.6s, T/S = 41.3
## M1  // Runtime:


In [ ]:
output_dict = {}

In [ ]:
# Generate the output
# name = "localized_deployment.txt"
for name in inputs_dict.keys():
    inputs = inputs_dict[name]

    if os.path.exists(f'data/embeddings/{name}.pt'):
        continue
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        output_dict[name] = outputs
        torch.save(outputs.hidden_states[-1], f'data/embeddings/{name}.pt')

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="2fabebc4-406d-4935-8dbf-6952ce2d0e41")
index = pc.Index("inftest")

In [ ]:
vectors = []
for embedding_file in os.listdir('data/embeddings'):
    print(f'Embedding: {embedding_file}, Size: {os.path.getsize(f"data/embeddings/{embedding_file}")}')
    name = embedding_file[:-3]
    metadata = {"name": name, "url": name_to_url[name]}
    tensor = torch.load(f'data/embeddings/{embedding_file}')
    vectors.append((name, tensor.numpy().tolist(), metadata))


In [ ]:
print(vectors[0])

In [ ]:
index.upsert(vectors=[vectors[0]])